In [1]:
import pandas as pd 
import numpy as np
import requests
import re
import bs4
from iteration_utilities import duplicates
from dotenv import load_dotenv
import os
import src.img_web_scraping as imgscr
import src.api_related_functions as apif
import src.data_set_fun as dataf


In [2]:
load_dotenv()
key = os.getenv("Potter_api_key")

### Dataset

In [3]:
df_ds= pd.read_csv('Input/Characters.csv',delimiter=';', skiprows=0 ,encoding='latin1')

In [4]:
df_ds['Name'] = df_ds['Name'].apply(dataf.nameshrink)

In [5]:
df_ds['Name'] = df_ds['Name'].replace('Lily Potter', 'Lily J. Potter' ).replace('James Potter','James Potter I')

#### Once the name column is  properly formated, we are going to get a list with all of the names in order to compare them with the names available in the api, so we create a full functional data frame.

In [6]:
lista_nombres_ds = [df_ds['Name'][x] for x in range (0,len(df_ds['Name']))]

In [7]:
print(list(duplicates(lista_nombres_ds)))
df_ds[df_ds['Name'].str.match('James Potter I')]

['James Potter I']


,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
12,13,James Potter I,Male,NaN,Gryffindor,"11"" Mahogany unknown core",Stag,Human,Pure-blood,Black,Hazel,Order of the Phoenix,Animagus| Seeker,"27 March, 1960","31 October, 1981"
127,128,James Potter I,Male,Student,Gryffindor,Unknown,Unknown,Human,Half-blood,NaN,NaN,NaN,NaN,1 September 2003-31 August 2004,NaN


In [8]:
df_ds = df_ds.drop([127])
df_ds = df_ds.reset_index(drop=True)

In [9]:
lista_nombres_ds = [df_ds['Name'][x] for x in range (0,len(df_ds['Name']))]
print(list(duplicates(lista_nombres_ds)))
df_ds.head()

[]


,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,NaN
1,2,Ronald Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,NaN
2,3,Hermione Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",NaN
3,4,Albus Dumbledore,Male,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,Silver| formerly auburn,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,Late August 1881,"30 June, 1997"
4,5,Rubeus Hagrid,Male,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",None,Half-Human/Half-Giant,Part-Human (Half-giant),Black,Black,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,6 December 1928,NaN


### Api Call
* For this proyect, I´m using the Potterapi, for further information please visit https://www.potterapi.com/

In [10]:
data = apif.call_api('characters',key)

In [11]:
data_api = apif.api_extraction(data)

In [12]:
df_api = pd.DataFrame(data_api)

In [13]:
df_api.head()

,name,role,species,bloodStatus,dumbledoresArmy,deathEater,ministryOfMagic,orderOfThePhoenix
0,Hannah Abbott,student,human,half-blood,True,False,False,False
1,Bathsheda Babbling,"Professor, Ancient Runes",human,unknown,True,False,False,False
2,Ludo Bagman,"Head, Department of Magical Games and Sports",human,unknown,False,False,True,False
3,Bathilda Bagshot,"Author, A History Of Magic",human,unknown,False,False,False,False
4,Katie Bell,student,human,pure-blood,False,False,False,False


In [14]:
lista_nombres_api=[df_api['name'][x] for x in range(0,len(df_api['name']))]

#### In order to have the same characters available, I´m going to find and drop the ones that are not in both data set and api.

In [15]:
drop_df_ds = [x for x in lista_nombres_ds if x not in lista_nombres_api]

In [16]:
idx_names_ds = [df_ds[ df_ds['Name'] == x ].index for x in drop_df_ds ]

In [17]:
[df_ds.drop(x, inplace = True) for x in idx_names_ds]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [18]:
df_ds = df_ds.reset_index(drop=True)

In [19]:
drop_df_api = [x for x in lista_nombres_api if x not in lista_nombres_ds]

In [20]:
idx_names_api = [df_api[ df_api['name'] == x ].index for x in drop_df_api ]

In [21]:
[df_api.drop(x, inplace = True) for x in idx_names_api]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [22]:
df_api = df_api.reset_index(drop=True)

#### I´m going to give a last review to the data frames and perform some modifications.

In [23]:
df_ds = df_ds.replace({'Death': r'.*'}, {'Death': True}, regex=True)
df_ds['Death'] = df_ds['Death'].fillna(False)

In [24]:
df_ds = df_ds[['Name', 'Gender','House','Loyalty','Skills','Death']]
df_ds = df_ds.fillna('Unknown')

In [25]:
df_ds = df_ds.sort_values('Name')

In [26]:
df_api = df_api.sort_values('name')

In [27]:
df_api.columns = df_api.columns.str.replace("role", "Role").str.replace("species", "Species").str.replace("bloodStatus", "Blood_Status").str.replace("dumbledoresArmy", "Dumbledores_Army").str.replace("deathEater", "Death_Eater").str.replace("ministryOfMagic", "Ministry_Of_Magic").str.replace("orderOfThePhoenix", "Order_Of_The_Phoenix")

In [28]:
df_api = df_api.reset_index(drop=True)
df_ds = df_ds.reset_index(drop=True)

#### Now I´m going to merge the final df_ds with the information extracted from the api and the images url from webscraping in https://harrypotter.fandom.com/wiki/Main_Page.

In [29]:
col_name_api=['Role','Species','Blood_Status','Dumbledores_Army','Death_Eater','Ministry_Of_Magic','Order_Of_The_Phoenix']

In [30]:
for x in col_name_api:
    df_ds[x] = df_api[x]

In [31]:
df_ds=df_ds.merge(df_api[col_name_api])

In [32]:
potter = df_ds

In [33]:
names_potter = [potter['Name'][x] for x in range(0,len(potter['Name']))]

In [34]:
names_potter = [x.replace('Lily J. Potter','Lily Evans' ) for x in names_potter]

In [35]:
links_img = [imgscr.scrap_img(x) for x in names_potter]

In [36]:
potter['Picture'] = pd.DataFrame(links_img)

In [37]:
potter.head()

,Name,Gender,House,Loyalty,Skills,Death,Role,Species,Blood_Status,Dumbledores_Army,Death_Eater,Ministry_Of_Magic,Order_Of_The_Phoenix,Picture
0,Aberforth Dumbledore,Male,Unknown,Order of the Phoenix,"Bartending, caring for Ariana",False,"Owner, Hog's Head Inn",human,half-blood,False,False,False,True,https://vignette.wikia.nocookie.net/esharrypot...
1,Alastor Moody,Male,Unknown,Order of the Phoenix,A highly accomplished Auror and an outstanding...,True,Unknown,human,pure-blood,False,False,True,True,https://vignette.wikia.nocookie.net/esharrypot...
2,Albus Dumbledore,Male,Gryffindor,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,True,"Headmaster, Hogwarts",human,half-blood,False,False,True,True,https://vignette.wikia.nocookie.net/esharrypot...
3,Alecto Carrow,Female,Slytherin,Lord Voldemort | Death Eaters,Spreading anti-Muggle propaganda,False,"Professor, Muggle Studies",human,pure-blood,False,True,False,False,https://vignette.wikia.nocookie.net/esharrypot...
4,Alice Longbottom,Female,Unknown,Original Order of the Phoenix,Auror,False,Auror (formerly),human,pure-blood,False,False,False,True,https://vignette.wikia.nocookie.net/esharrypot...


In [38]:
potter.to_csv('potter.csv', index=False)